In [144]:
from collections import OrderedDict
import torch

import numpy as np
from torchvision.models import resnet18

In [145]:
ckpt=torch.load('resnet18-f37072fd.pth')

In [146]:
m=resnet18(pretrained=True)
m.load_state_dict(ckpt)
_=m.eval()
torch.set_grad_enabled(False)

/home/lyan/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lyan/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [170]:
def write_graph(model, output_name):
#     submodules=list(model.modules())
    
    f = open(output_name, 'wb')
    n=3
    
    header = np.zeros(256, dtype=np.int32)
    header[0]=42
    header[1]=1
    header[2] = n

    f.write(header.tobytes())
    offset=256
    for i, l in enumerate(model.modules()):
        print(i,type(l), offset)
            
        if type(l) == torch.nn.Conv2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=1 # type for conv
            header[1]=l.in_channels; header[2]=l.out_channels
            header[3]=l.kernel_size[0]; header[4]=l.kernel_size[1]
            header[5]=l.stride[0];header[6]=l.stride[1]
            
            header[7]=l.padding[0];header[8]=l.padding[1]
            if l.bias is None:
                header[9]=0
            else:
                header[9]=int(l.bias)
            header[10]=l.groups
            header[11]=l.dilation[0];header[12]=l.dilation[1];
            
            weight=l.weight.detach().to(torch.float32).numpy()
            for i in range(len(weight.shape)):
                header[13+i] = weight.shape[i]

            offset += len(header)
            offset += len(weight.reshape(-1))
            print(weight.reshape(-1)[0:10])
            f.write(header.tobytes())
            f.write(weight.tobytes())
            if l.bias is not None or l.bias:
                f.write(l.bias.detach().to(torch.float32).numpy().tobytes())
            
        elif type(l) == torch.nn.BatchNorm2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=2 # type for conv
            header[1] = l.num_features
            
            f.write(header.tobytes())
            f.write(l.weight.detach().to(torch.float32).numpy().tobytes())
            f.write(l.bias.detach().to(torch.float32).numpy().tobytes())
            f.write(l.running_mean.detach().to(torch.float32).numpy().tobytes())
            f.write(l.running_var.detach().to(torch.float32).numpy().tobytes())
            offset += 256
            offset += l.weight.shape[0]*4
        elif type(l) == torch.nn.ReLU:
            header = np.zeros(256, dtype=np.int32)
            header[0]=3 # type for conv
            f.write(header.tobytes())
            offset+=256
        elif type(l) == torch.nn.MaxPool2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=4
            header[1]=l.kernel_size
            header[2]=l.stride
            header[3]=l.padding
            header[4]=l.dilation
            header[5] = 102
            f.write(header.tobytes())
            offset+=256
        elif type(l) == torch.nn.AdaptiveAvgPool2d:
            header = np.zeros(256, dtype=np.int32)
            header[0]=5
            header[1]=l.output_size[0]
            header[2]=l.output_size[1]
            f.write(header.tobytes())
        elif type(l) == torch.nn.Linear:
            header = np.zeros(256, dtype=np.int32)
            header[0]=6
            header[1]=l.in_features
            header[2]=l.out_features
            f.write(header.tobytes())
            f.write(l.weight.detach().to(torch.float32).numpy().tobytes())
            f.write(l.bias.detach().to(torch.float32).numpy().tobytes())
        else:
#             print(type(l))
            continue
    f.close()

In [171]:
write_graph(m, 'r18.bin')

0 <class 'torchvision.models.resnet.ResNet'> 256
1 <class 'torch.nn.modules.conv.Conv2d'> 256
[-0.01041935 -0.00613561 -0.00180978  0.07484142  0.05661485  0.01708333
 -0.01269388  0.01108271  0.00952757 -0.10992692]
2 <class 'torch.nn.modules.batchnorm.BatchNorm2d'> 9920
3 <class 'torch.nn.modules.activation.ReLU'> 10432
4 <class 'torch.nn.modules.pooling.MaxPool2d'> 10688
5 <class 'torch.nn.modules.container.Sequential'> 10944
6 <class 'torchvision.models.resnet.BasicBlock'> 10944
7 <class 'torch.nn.modules.conv.Conv2d'> 10944
[ 0.05759342 -0.09511436 -0.02027232 -0.07455588 -0.799308   -0.21283598
  0.06557069 -0.09653367 -0.01211061 -0.00699444]
8 <class 'torch.nn.modules.batchnorm.BatchNorm2d'> 48064
9 <class 'torch.nn.modules.activation.ReLU'> 48576
10 <class 'torch.nn.modules.conv.Conv2d'> 48832
[ 0.02594677 -0.10457563 -0.00477124 -0.08622317 -0.33020768 -0.10275265
 -0.05742571 -0.19074456 -0.05464623 -0.01695126]
11 <class 'torch.nn.modules.batchnorm.BatchNorm2d'> 85952
12 <c

In [201]:
def write_data(x, name, output_name):
    with open(output_name, 'wb') as f:
        header = np.zeros(256, dtype=np.int32)
        header[0]=42
        header[1]=1
        for i in range(len(x.shape)):
            header[2+i] = x.shape[i]
        
        f.write(header.tobytes())
        
        f.write(x.tobytes())

In [5]:
import cv2

In [6]:
img=cv2.imread('/home/lyan/Downloads/000000062491.jpg')
# img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img=cv2.resize(img, (64,64))
img=torch.tensor(img).reshape(1,3,64,64).to(torch.float32) / 255

In [189]:
img.reshape(-1)[0:10]

tensor([0.4039, 0.5255, 0.5216, 0.4549, 0.5412, 0.5451, 0.4667, 0.5725, 0.5529,
        0.4784])

In [149]:
with open('r18.bin','rb') as f:
    data=f.read()

In [150]:
off=10944*4+256*4

In [151]:
np.frombuffer(data[off : off+32*4], dtype=np.float32)

array([ 5.7593420e-02, -9.5114358e-02, -2.0272322e-02, -7.4555881e-02,
       -7.9930800e-01, -2.1283598e-01,  6.5570690e-02, -9.6533671e-02,
       -1.2110611e-02, -6.9944421e-03,  1.4265955e-02,  5.5824057e-04,
        4.1238047e-02, -1.6125326e-01, -2.3207795e-02,  3.2886832e-03,
        7.1778586e-03,  7.1686357e-02, -2.3627296e-09, -3.9269693e-08,
       -3.2971212e-08,  2.1736867e-08,  8.3299465e-09,  1.2542622e-08,
        1.1381505e-08,  8.8095797e-09,  1.5505723e-08,  1.2640991e-03,
       -5.1235743e-03,  6.9588250e-03, -2.1271452e-02, -6.8267517e-02],
      dtype=float32)

In [181]:
x=m.conv1(img)
x=m.bn1(x)
x=m.relu(x)

In [182]:
x2=m.maxpool(x)

In [191]:
x2.reshape(-1)[0:10]

tensor([0.2977, 0.3026, 0.3189, 0.3811, 0.3867, 0.3234, 0.2848, 0.3312, 0.3312,
        0.3210])

In [204]:
x3=m.layer1[0].conv1(x2)
x3=m.layer1[0].bn1(x3)
x3=m.layer1[0].relu(x3)

In [177]:
m.layer1[0].conv1.weight.data.reshape(-1)[0:20]

tensor([ 5.7593e-02, -9.5114e-02, -2.0272e-02, -7.4556e-02, -7.9931e-01,
        -2.1284e-01,  6.5571e-02, -9.6534e-02, -1.2111e-02, -6.9944e-03,
         1.4266e-02,  5.5824e-04,  4.1238e-02, -1.6125e-01, -2.3208e-02,
         3.2887e-03,  7.1779e-03,  7.1686e-02, -2.3627e-09, -3.9270e-08])

In [205]:
x3.reshape(-1)[0:30]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0817,
        0.0183, 0.0105, 0.0006, 0.1063, 0.1492, 0.1358, 0.1691, 0.1527, 0.1526,
        0.1763, 0.2330, 0.2611])

In [215]:
x2.shape

torch.Size([1, 64, 16, 16])

In [213]:
x2.reshape(-1)[0:512]

tensor([0.2977, 0.3026, 0.3189, 0.3811, 0.3867, 0.3234, 0.2848, 0.3312, 0.3312,
        0.3210, 0.2436, 0.2212, 0.2146, 0.1921, 0.2224, 0.2475, 0.4758, 0.4646,
        0.5618, 0.5693, 0.5693, 0.4825, 0.4603, 0.4603, 0.4437, 0.4504, 0.4504,
        0.4118, 0.3566, 0.3365, 0.3461, 0.3511, 0.5473, 0.4906, 0.4269, 0.4824,
        0.4824, 0.4569, 0.3510, 0.3281, 0.3281, 0.3523, 0.4547, 0.4776, 0.4500,
        0.4196, 0.3550, 0.3259, 0.5473, 0.4906, 0.4359, 0.4907, 0.4903, 0.3782,
        0.3782, 0.2928, 0.2608, 0.3523, 0.4547, 0.4776, 0.4500, 0.4196, 0.3550,
        0.3259, 0.5068, 0.4746, 0.4604, 0.4907, 0.4903, 0.3782, 0.3782, 0.3065,
        0.2702, 0.3848, 0.4015, 0.4297, 0.4284, 0.4199, 0.4025, 0.3711, 0.4297,
        0.4297, 0.4944, 0.4935, 0.4935, 0.3139, 0.3336, 0.3336, 0.2817, 0.3976,
        0.4336, 0.4452, 0.4532, 0.4424, 0.4266, 0.3832, 0.4297, 0.4378, 0.4944,
        0.4935, 0.4935, 0.3273, 0.3336, 0.3336, 0.2895, 0.3976, 0.4336, 0.4452,
        0.4532, 0.4424, 0.4266, 0.3832, 

In [35]:
import matplotlib.pyplot as plt

In [203]:
write_data(x2.numpy(), '', 'x2.bin')

In [170]:
write_data(img.numpy(), '', 'test_img.bin')
write_data(ckpt['conv1.weight'].detach().to(torch.float32).numpy(), 'conv1.weight', 'test.bin')

In [202]:
y=torch.nn.functional.conv2d(img, ckpt['conv1.weight'].detach() , )